<a href="https://colab.research.google.com/github/Liza-IITP/Dynamic_Pricing_Parking_Lots/blob/main/trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------ INSTALLATION ------------------
!pip install ydata-profiling --quiet
!pip install pathway bokeh --quiet


In [ ]:
# ------------------ IMPORTS ------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

In [ ]:

# ------------------ CSV LOAD + PROFILE REPORT ------------------
df = pd.read_csv('/content/Modified - modified.csv')
from ydata_profiling import ProfileReport
report = ProfileReport(df)
report.to_file(output_file='report.html')

In [ ]:

# ------------------ FEATURE ENGINEERING ------------------
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")
vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
df = pd.concat([df, vehicle_dummies], axis=1)
df.drop(columns=["VehicleType"], inplace=True)

traffic_map = {'low': 0, 'average': 1, 'high': 2}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

df.drop(columns=["Unnamed: 0", "SystemCodeNumber", "Capacity", "Occupancy",
                 "Latitude", "Longitude", "LastUpdatedDate", "LastUpdatedTime", "TrafficConditionNearby"],
        inplace=True)

df["DayOfWeek"] = df["Timestamp"].dt.dayofweek
df.drop_duplicates(inplace=True)
df = df.sort_values('Timestamp').reset_index(drop=True)

print(df.head())
df.to_csv("parking_stream.csv", index=False)
print(df.columns.tolist())

In [ ]:

# ------------------ ALPHA ESTIMATION (from scratch) ------------------
alpha_dummy = 1.0
base_price = 10
price_series = [base_price]
for occ in df["OccupancyRate"]:
    price_series.append(price_series[-1] + alpha_dummy * occ)
df["SimulatedPrice"] = price_series[1:]

df["DeltaPrice"] = df["SimulatedPrice"].diff()
df["OccupancyRate_lag"] = df["OccupancyRate"].shift(1)
df_clean = df.dropna(subset=["DeltaPrice", "OccupancyRate_lag"])

x = df_clean["OccupancyRate_lag"].values
y = df_clean["DeltaPrice"].values
x_mean = x.mean()
y_mean = y.mean()
numerator = ((x - x_mean) * (y - y_mean)).sum()
denominator = ((x - x_mean) ** 2).sum()
alpha = numerator / denominator

print(f"📈 Estimated alpha (from scratch): {alpha:.4f}")

In [ ]:

# ------------------ PATHWAY STREAM SETUP ------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    OccupancyRate: float
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_cycle: int
    VehicleType_truck: int

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:

# ------------------ MODEL 1: Alpha-Based Pricing ------------------
model1_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        total_occ=pw.reducers.sum(pw.this.OccupancyRate),
    )
    .with_columns(
        Price_Model1 = 10 + alpha * pw.this.total_occ
    )
)

def plot_model1(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1: Linear Dynamic Price Based on Occupancy (Estimated α)",
        x_axis_type="datetime"
    )
    fig.line("t", "Price_Model1", source=source, line_width=2, color="blue")
    fig.scatter("t", "Price_Model1", source=source, size=6, color="red")
    return fig

viz_model1 = model1_window.plot(plot_model1, sorting_col="t")
pn.Column(viz_model1).servable()



In [ ]:
# ------------------ MODEL 2: Fluctuation-Based Pricing ------------------
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.OccupancyRate),
        occ_min=pw.reducers.min(pw.this.OccupancyRate),
    )
    .with_columns(
        Price = 10 + (pw.this.occ_max - pw.this.occ_min)
    )
)

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price (Model 2 - Fluctuation Based)",
        x_axis_type="datetime"
    )
    fig.line("t", "Price", source=source, line_width=2, color="navy")
    fig.scatter("t", "Price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

In [ ]:
# ------------------ RUN ------------------
%%capture --no-display
pw.run()